In [3]:
!pip3 install git+https://github.com/tira-io/tira.git@development#\&subdirectory=python-client

  Cloning https://github.com/tira-io/tira.git (to revision development) to /tmp/pip-req-build-ru2ybaxv
  Running command git clone --filter=blob:none --quiet https://github.com/tira-io/tira.git /tmp/pip-req-build-ru2ybaxv
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Resolved https://github.com/tira-io/tira.git to commit 7dd0e7216943ad2844b62c36939b60920e034a2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tira: filename=tira-0.0.134-py3-none-any.whl size=94772 sha256=c0c5a6a0cb2c523284225a86f7229034536adef761bbc6d2b17267ca68933a94
  Stored in directory: /tmp/pip-ephem-wheel-cache-dkempz50/wheels/1f/36/7a/0f641175add91ee2acac0e59ab6ed9b3b04069e89b555425a4
Successfully built tira


In [1]:
from tira.rest_api_client import Client
import pandas as pd

tira = Client()

In [2]:
APPROACHES = [
    # tinyfsu
    'reneuir-2024/tinyfsu/tiny-fsu-bert',
    
    #'cross-encoder-ms-marco-TinyBERT-L-2',
    
    #tira ir starter
    'reneuir-2024/naverlabseurope/Splade (re-ranker)',
    'reneuir-2024/tira-ir-starter/MonoT5 Base (tira-ir-starter-gygaggle)',
    'reneuir-2024/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)',
    'reneuir-2024/tira-ir-starter/TASB msmarco-distilbert-base-cos (tira-ir-starter-beir)',
    'reneuir-2024/tira-ir-starter/ANCE Base Dot (tira-ir-starter-beir)',
    'reneuir-2024/tira-ir-starter/MonoT5 3b (tira-ir-starter-gygaggle)',
    'reneuir-2024/tira-ir-starter/MonoBERT Large (tira-ir-starter-gygaggle)',
    #'reneuir-2024/tira-ir-starter/MonoBERT Small (tira-ir-starter-gygaggle)',
    #SBERT multi-qa-mpnet-base-dot-v1 (tira-ir-starter-beir)
    #SBERT msmarco-distilbert-dot-v5 (tira-ir-starter-beir)
    
    # fschlatt
    'reneuir-2024/fschlatt/sparse-cross-encoder-4-512',
    'reneuir-2024/fschlatt/castorini-list-in-t5-150',
    'reneuir-2024/fschlatt/rank-zephyr',
    # TODO: monoelectra
    
    
    # Sean
    # TODO: Approach presented from workshop
    
    # Eugene
    # TODO: Approach presented from workshop
    
    # Shengyao
    # ToDO Approach presented from workshop
    
    # 
    # 6x query expansion with Llama Flant5 >> BM25
]

In [4]:
df = []

for approach in APPROACHES:
    profiling = tira.profiling.from_submission(approach, 'dl-top-1000-docs-20240701-training')
    elapsed_time = profiling[-1]
    assert elapsed_time['key'] == 'elapsed_time'
    elapsed_time = elapsed_time['value']
    rss = [i['value'] for i in profiling if i['key'] == 'ps_vsz']
    gpu_memory_used = [int(i['value'].split()[0]) for i in profiling if i['key'] == 'gpu_memory_used']
    gpu_utilization = [int(i['value'].split()[0]) for i in profiling if i['key'] == 'gpu_utilization']
    cpu_utilization = [i['value'] for i in profiling if i['key'] == 'ps_cpu']
    df += [{
        'approach': approach,
        'Elapsed Time': elapsed_time,
        'CPU Utilization (Max)': max(cpu_utilization),
        'RSS (MAX)': max(rss),
        'GPU Memory (Max)': max(gpu_memory_used),
        'GPU Utilization (Max)': max(gpu_utilization),
    }]

df = pd.DataFrame(df)
df.sort_values('Elapsed Time')

,approach,Elapsed Time,CPU Utilization (Max),RSS (MAX),GPU Memory (Max),GPU Utilization (Max)
0,reneuir-2024/tinyfsu/tiny-fsu-bert,64.0,135.7,17957372.0,250,23
4,reneuir-2024/tira-ir-starter/TASB msmarco-dist...,243.0,121.1,23648928.0,6380,100
8,reneuir-2024/fschlatt/sparse-cross-encoder-4-512,360.0,143.2,394142648.0,3238,100
5,reneuir-2024/tira-ir-starter/ANCE Base Dot (ti...,462.0,120.1,23146064.0,5880,100
3,reneuir-2024/tira-ir-starter/ColBERT Re-Rank (...,910.0,113.4,27281316.0,2350,100
1,reneuir-2024/naverlabseurope/Splade (re-ranker),1854.0,116.0,25973608.0,1386,62
7,reneuir-2024/tira-ir-starter/MonoBERT Large (t...,2094.0,148.4,45561864.0,2622,94
2,reneuir-2024/tira-ir-starter/MonoT5 Base (tira...,2127.0,172.0,47311400.0,2528,85
10,reneuir-2024/fschlatt/rank-zephyr,2530.0,128.4,62081316.0,22902,100
9,reneuir-2024/fschlatt/castorini-list-in-t5-150,2614.0,180.1,122621104.0,6850,100
